# Grover's Algorithm

The **Grover's Search** quantum kata is a series of exercises designed
to get you familiar with Grover's search algorithm.

It covers the following topics:

* writing oracles for Grover's search,
* performing steps of the algorithm, and
* putting it all together: Grover's search algorithm.

*Reading material:*

* The tasks follow the explanation from *Quantum Computation and Quantum Information* by Nielsen and Chuang.
  In the 10th anniversary edition, this is section 6.1.2 on pages 248-251.
* A different explanation of Grover's algorithm can be found in 
  [this Wikipedia article](https://en.wikipedia.org/wiki/Grover%27s_algorithm).
* [An Introduction to Quantum Algorithms](https://people.cs.umass.edu/~strubell/doc/quantum_tutorial.pdf) by Emma Strubell, pages 20-24.
* [Lecture 4: Grover's Algorithm](https://www.cs.cmu.edu/~odonnell/quantum15/lecture04.pdf) by John Wright.
* Lectures [12](https://cs.uwaterloo.ca/~watrous/LectureNotes/CPSC519.Winter2006/12.pdf) and [13](https://cs.uwaterloo.ca/~watrous/LectureNotes/CPSC519.Winter2006/13.pdf) by John Watrous.
* [This page](http://davidbkemp.github.io/animated-qubits/grover.html) has an animated demonstration of Grover's algorithm for a simple case.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blanks (marked with the `// ...` comments)
with some Q# code that solves the task.  To verify your answer, run the cell with Ctrl/⌘+Enter.

Within each section, tasks are given in approximate order of increasing difficulty;
harder ones are marked with asterisks.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [1]:
%package Microsoft.Quantum.Katas::0.11.2004.2825

Adding package Microsoft.Quantum.Katas::0.11.2004.2825: done!

Microsoft.Quantum.Standard::0.11.2004.2825, Microsoft.Quantum.Katas::0.11.2004.2825

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Oracles for Grover's Search


### Task 1.1. The $|11...1\rangle$ Oracle
**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Flip the state of the target qubit (i.e., apply an X gate to it)
if the query register is in the $|11...1\rangle$ state,
and leave it unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Examples:**

* If the query register is in state $|00...0\rangle$, leave the target qubit unchanged.

* If the query register is in state $|10...0\rangle$, leave the target qubit unchanged.

* If the query register is in state $|11...1\rangle$, flip the target qubit.

* If the query register is in state $\frac{1}{\sqrt{2}} \big(|00...0\rangle + |11...1\rangle \big)$, and the target is in state $|0\rangle$,
the joint state of the query register and the target qubit should be $\frac{1}{\sqrt{2}} \big(|00...00\rangle + |11...11\rangle \big)$.

Remember that CNOT on a **register** of qubits checks that ALL qubits are in |1> 

In [2]:
%kata T11_Oracle_AllOnes_Test 

open Microsoft.Quantum.Diagnostics;

operation Oracle_AllOnes (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    Controlled X(queryRegister, target);
}

Success!

### Task 1.2. The $|1010...\rangle$ Oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

  Flip the state of the target qubit if the query register is in the $|1010...\rangle$ state;
that is, the state with alternating 1 and 0 values, with any number of qubits in the register.
Leave the state of the target qubit unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Examples:**

 * If the register is in state $|0000000\rangle$, leave the target qubit unchanged.
 * If the register is in state $|10101\rangle$, flip the target qubit.

If you follow the goal explicitly, you can notice that you are guranteed that the 0th qubit is a 1<br>
.:. flipping the "odd" positions will put the register in the |1...1> state<br>
also notice that you can make this observation work on a conditional<br> 
*measure the first qubit<br>
if it's a 0 then you flip the evens*

1. Learned the syntax for range(start, end, step) in Q#!
2. `within{} apply{}` is called **conjugations**<br>
according to the `quickref.pdf`, it's applying $ABA^{T}$<br>
**NB** i'm not sure which operators A and B would be in this case

In [12]:
%kata T12_Oracle_AlternatingBits_Test 

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(queryRegister);
    within{
        for (i in 1..2..N-1){
            X(queryRegister[i]);
        }
    } apply{
        Controlled X(queryRegister, target);
    }
}

Success!

### Task 1.3. Arbitrary Bit Pattern Oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. A bit pattern of length N represented as `Bool[]`

**Goal:**

  Flip the state of the target qubit if the query register is in the state described by the given bit pattern
(`true` represents qubit state One, and `false` represents Zero).
Leave the state of the target qubit unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Example:**

  If the bit pattern is `[true, false]`, you need to flip the target qubit if and only if the qubits are in the $|10\rangle$ state.

In [15]:
%kata T13_Oracle_ArbitraryPattern_Test 

operation Oracle_ArbitraryPattern (queryRegister : Qubit[], target : Qubit, pattern : Bool[]) : Unit is Adj {
    (ControlledOnBitString(pattern, X))(queryRegister, target);
}

Success!

### Task 1.4. Oracle Converter

**Input:**

A marking oracle: an oracle that takes a register and a target qubit and
flips the target qubit if the register satisfies a certain condition.

**Output:**

A phase-flipping oracle: an oracle that takes a register and
flips the phase of the register if it satisfies this condition.

> Grover's algorithm relies on the search condition implemented as a phase-flipping oracle,
but it is often easier to write a marking oracle for a given condition. This transformation
allows to convert one type of oracle into the other. The transformation is described in the 
[Wikipedia article on Grover's algorithm](https://en.wikipedia.org/wiki/Grover%27s_algorithm), in the section "Description of ${U_\omega}$".

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
    Remember that you can define auxiliary operations. To do that, you'll need to create an extra code cell for each new operation and execute it before returning to this cell.
</details>

1. the **phase kickback** trick is being used here => need to study this topic
2. the extra operation that we had to implement was just an implementation of the first steps of grover's alg
    - why is this the correct thing to do?
3. the `witihin{} apply{}` explained:
    - saying do the body of `apply` without releasing the state of the qubits `within`
    - not clear how this is different from applying all gates *inside of* the `using` body

In [18]:
operation ApplyMarkingOracle (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    using (target=Qubit()){
        within{
            X(target);
            H(target);
        }
        apply{
            markingOracle(register, target);
        }
    }
}

ApplyMarkingOracle

In [19]:
%kata T14_OracleConverter_Test 

function OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    return ApplyMarkingOracle(markingOracle, _);
}

Success!

## Part II. The Grover Iteration

### Task 2.1. The Hadamard Transform

**Input:** A register of N qubits in an arbitrary state

**Goal:** Apply the Hadamard transform to each of the qubits in the register.

> If the register started in the $|0...0\rangle$ state, this operation
will prepare an equal superposition of all $2^{N}$ basis states.

In [20]:
%kata T21_HadamardTransform_Test 

operation HadamardTransform (register : Qubit[]) : Unit is Adj {
    for (qubit in register){
        H(qubit);
    }
}

Success!

### Task 2.2. Conditional Phase Flip

**Input:**  A register of N qubits in an arbitrary state.

**Goal:**  Flip the sign of the state of the register if it is not in the $|0...0\rangle$ state.

**Examples:**

 * If the register is in state $|0...0\rangle$, leave it unchanged.

 * If the register is in any other basis state, multiply its phase by -1.

> This operation implements operator $2|0...0\rangle\langle0...0| - I$ $ = \left(\begin{matrix}1&0&...&0\\0&-1&...&0\\\vdots&\vdots&\ddots&\vdots\\0&0&...&-1\end{matrix}\right) $

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
    Note that quantum states are defined up to a global phase.
    Thus the state obtained as a result of this operation is equivalent to the state obtained by flipping the sign of only the $|0...0\rangle$ basis state (those states differ by a global phase -1).<br>
    $$-\big(2|0...0\rangle\langle0...0| - I\big) = I - 2|0...0\rangle\langle0...0| = \left(\begin{matrix}-1&0&...&0\\0&1&...&0\\\vdots&\vdots&\ddots&\vdots\\0&0&...&1\end{matrix}\right) $$<br>
</details>
<br/>

<details>
  <summary><b>Need another hint? Click here</b></summary>
    Consider the Controlled Z gate, applied with most of the qubits as control and the last qubit as target:
    $\text{Controlled Z}(|s_0 s_1 \ldots s_{n-2}\rangle, |s_{n-1}\rangle)$ leaves all basis states except $|1...11\rangle$ unchanged, and adds a $-1$ phase to that state: $|1...11\rangle \rightarrow -|1...11\rangle$ (remember that $Z|0\rangle = |0\rangle$ and $Z|1\rangle = -|1\rangle$). 
    You need to modify it to add the $-1$ phase to only the $|0...00\rangle$ state instead.
    <br/><br/>
    Alternatively, you can use the same trick as in the oracle converter task.<br>
</details>

1. definitely understood that using a Z gate would add the necessary global phase
2. this operation works on the basis that the phase being changed is the phase of |0...0>
    - all states are defined only up to a global phase (because global phase is not observable)
    - so we can think about grover's algorithm working to change |x> phase
    - **OR** we can think about it **changing the phase of |0...0>**
    - **THE END RESULT IS THE SAME**
3. in the code itself
    - need to study conjugates more
    - learned about Arrays methods `Most` and `Tail` (could be convenient)
4. **the new goal is to flip the phase of the |0...0>**
    - if we have |0...0> as input...
    - take advantage of controlled gates by doing a CZ on the Tail qubit
        - NB we only want to introduce 1 phase flip at most; that's why we don't flip on every instance of One

In [27]:
%kata T22_ConditionalPhaseFlip_Test 

open Microsoft.Quantum.Arrays;

operation ConditionalPhaseFlip (register : Qubit[]) : Unit is Adj {
    
    within{
    ApplyToEachA(X, register);
    } apply{
    (Controlled Z)(Most(register), Tail(register));
    }
    
    
}

Success!

### Task 2.3. The Grover Iteration

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A phase-flipping oracle that takes an N-qubit register and flips
     the phase of the state if the register is in the desired state.

**Goal:**  Perform one Grover iteration.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
    A Grover iteration consists of 4 steps:
    <ol>
    <li>Apply the Oracle</li>
    <li>Apply the Hadamard transform</li>
    <li>Perform a conditional phase shift</li>
    <li>Apply the Hadamard transform again</li>
    </ol>
</details>

In [30]:
%kata T23_GroverIteration_Test 

operation GroverIteration (register : Qubit[], oracle : (Qubit[] => Unit is Adj)) : Unit is Adj {
    oracle(register);
    ApplyToEachA(H, register);
    ConditionalPhaseFlip(register);
    ApplyToEachA(H, register);
}

Success!

## Part III. Putting It All Together: Grover's Search Algorithm

### Task 3.1. Grover's Search

**Inputs:**

  1. N qubits in the $|0...0\rangle$ state.

  2. A marking oracle.

  3. The number of Grover iterations to perform.

**Goal:** Use Grover's algorithm to leave the register in the state that is marked by the oracle as the answer (with high probability).

> The number of iterations is passed as a parameter because it is defined by the nature of the problem
and is easier to configure/calculate outside the search algorithm itself (for example, in the classical driver).

1. Why do i have to `ApplyToEach(H, register);` before entering the for loop? Doesn't `GroverIteration` handle that?
2. from notes:
    - GroverIteration takes inputs:  `Qubit[]`, `PHASE ORACLE`
    - Given in problem:  `Qubit[]`, `MARKING ORACLE`
    - *know*:  `OracleConverter(...)` brings `MARKING ORACLE` to `PHASE ORACLE`
3. notice that the for loop starts at the 1th qubit
    - does that mean the 0th qubit is the 

In [46]:
%kata T31_GroversSearch_Test 

operation GroversSearch (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    
    let phaseOracle = OracleConverter(oracle);
    HadamardTransform(register);
    for (i in 1..iterations){

        GroverIteration(register, phaseOracle);

    }
}

Success!

### Task 3.2. Using Grover's Search

**Goal:**   Use your implementation of Grover's Algorithm from Task 3.1 and the oracles from part 1
  to find the marked elements of the search space. This task is not covered by a test and allows you to experiment with running the algorithm.
  
> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details closed>
  <summary><b>Hint #1</b></summary>
    To check whether the algorithm found the correct answer (i.e., an answer marked as 1 by the oracle),
    you can apply the oracle once more to the register after you've measured it and an ancilla qubit,
    which will calculate the function of the answer found by the algorithm.
</details>
<br/>
<details closed>
  <summary><b>Hint #2</b></summary>
    Experiment with the number of iterations to see how it affects
    the probability of the algorithm finding the correct answer.
</details>
<br/>
<details closed>
  <summary><b>Hint #3</b></summary>
    You can use the Message function to output the results.
</details>

In [ ]:
operation Run_GroversSearch_Algorithm () : Unit {
    // ...
}

In [ ]:
%simulate Run_GroversSearch_Algorithm